# 추천시스템 구현해보기

In [1]:
import pandas as pd

In [39]:
user = pd.read_csv('users.dat', sep='::', header=None, names=['UserID','Gender', 'Age', 'Occupation', 'Zip-code'])
#처음부터 names 불러와서 보는 방법

C:\Users\user\AppData\Local\Temp\ipykernel_22200\3508904308.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  user = pd.read_csv('users.dat', sep='::', header=None, names=['UserID','Gender', 'Age', 'Occupation', 'Zip-code'])


In [40]:
user.rename(columns={0:'UserID'})

,UserID,Gender,Age,Occupation,Zip-code
0,1,F,1,10,48067
1,2,M,56,16,70072
2,3,M,25,15,55117
3,4,M,45,7,02460
4,5,M,25,20,55455
...,...,...,...,...,...
6035,6036,F,25,15,32603
6036,6037,F,45,1,76006
6037,6038,F,56,1,14706
6038,6039,F,45,0,01060


## 이번에는 불러들인 다음에 컬럼명 바꾸는 방식

In [19]:
rating = pd.read_csv('ratings.dat', sep='::', header=None)

C:\Users\user\AppData\Local\Temp\ipykernel_22200\3336608182.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  rating = pd.read_csv('ratings.dat', sep='::', header=None)


In [25]:
rating.rename(columns={0:'UserID', 1:'MovieID',2:'Rating',3:'Timestamp'}, inplace=True)

In [26]:
rating

,UserID,MovieID,Rating,Timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291
...,...,...,...,...
1000204,6040,1091,1,956716541
1000205,6040,1094,5,956704887
1000206,6040,562,5,956704746
1000207,6040,1096,4,956715648


## movie.dat 불러오기

In [27]:
movie = pd.read_csv('movies.dat')
#이렇게 부르면, UnicodeDecodeError 가 생긴다.

UnicodeDecodeError: 'utf-8' codec can't decode byte 0xe9 in position 3114: invalid continuation byte

In [35]:
movie = pd.read_csv('movies.dat', encoding='latin1', sep='::', header=None )
# rating = pd.read_csv('ratings.dat', sep='::', header=None)

C:\Users\user\AppData\Local\Temp\ipykernel_22200\1275947296.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  movie = pd.read_csv('movies.dat', encoding='latin1', sep='::', header=None )


In [37]:
movie.rename({0:'MovieID', 1: 'Title', 2:'Genres'}, axis=1, inplace=True)

In [38]:
movie

,MovieID,Title,Genres
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
3878,3948,Meet the Parents (2000),Comedy
3879,3949,Requiem for a Dream (2000),Drama
3880,3950,Tigerland (2000),Drama
3881,3951,Two Family House (2000),Drama


## join 시켜보기

In [42]:
rating.merge(movie).merge(user)

,UserID,MovieID,Rating,Timestamp,Title,Genres,Gender,Age,Occupation,Zip-code
0,1,1193,5,978300760,One Flew Over the Cuckoo's Nest (1975),Drama,F,1,10,48067
1,1,661,3,978302109,James and the Giant Peach (1996),Animation|Children's|Musical,F,1,10,48067
2,1,914,3,978301968,My Fair Lady (1964),Musical|Romance,F,1,10,48067
3,1,3408,4,978300275,Erin Brockovich (2000),Drama,F,1,10,48067
4,1,2355,5,978824291,"Bug's Life, A (1998)",Animation|Children's|Comedy,F,1,10,48067
...,...,...,...,...,...,...,...,...,...,...
1000204,4211,3791,2,965319075,Footloose (1984),Drama,M,45,5,77662
1000205,4211,3806,3,965319138,MacKenna's Gold (1969),Western,M,45,5,77662
1000206,4211,3840,4,965319197,Pumpkinhead (1988),Horror,M,45,5,77662
1000207,4211,3766,2,965319138,Missing in Action (1984),Action|War,M,45,5,77662


## collaborative recommendations 해보기

In [43]:
#우선 데이터를 찝어서 그에 관한 테이블 만들기 <- 피봇 활용
data=rating.merge(movie).merge(user)

In [44]:
data.pivot(index='UserID', columns='MovieID', values='Rating')

MovieID,1,2,3,4,5,6,7,8,9,10,...,3943,3944,3945,3946,3947,3948,3949,3950,3951,3952
UserID,,,,,,,,,,,,,,,,,,,,,
1,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6036,NaN,NaN,NaN,2.0,NaN,3.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6037,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6038,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### sci 테크닉 써보기

In [46]:
rating.set_index(['UserID', 'MovieID']).unstack()

Rating                                               ... Timestamp  \
MovieID   1    2    3    4    5    6    7    8    9    10    ...      3943   
UserID                                                       ...             
1          5.0  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  ...       NaN   
2          NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  ...       NaN   
3          NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  ...       NaN   
4          NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  ...       NaN   
5          NaN  NaN  NaN  NaN  NaN  2.0  NaN  NaN  NaN  NaN  ...       NaN   
...        ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...       ...   
6036       NaN  NaN  NaN  2.0  NaN  3.0  NaN  NaN  NaN  NaN  ...       NaN   
6037       NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  ...       NaN   
6038       NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  ...       NaN   
6039       NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  ...       NaN   
6040       3.0  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  ...       NaN   

                                                      
MovieID 3944 3945 3946 3947 3948 3949 3950 3951 3952  
UserID                                                
1        NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  
2        NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  
3        NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  
4        NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  
5        NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  
...      ...  ...  ...  ...  ...  ...  ...  ...  ...  
6036     NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  
6037     NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  
6038     NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  
6039     NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  
6040     NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  

[6040 rows x 7412 columns]

### NaN 을 채우고 싶다면?

In [47]:
user_item = data.pivot(index='UserID', columns='MovieID', values='Rating').fillna(0)

In [48]:
user_item

MovieID,1,2,3,4,5,6,7,8,9,10,...,3943,3944,3945,3946,3947,3948,3949,3950,3951,3952
UserID,,,,,,,,,,,,,,,,,,,,,
1,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6036,0.0,0.0,0.0,2.0,0.0,3.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6037,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6038,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [49]:
#1번 사람이 1번 영화에 평점 5점 준걸 한눈에 알아볼 수 있다.

#컬럼간에 co-relation 을 지원해주는걸 활용해볼거다.
#그 전에 행과 열을 바꿔줘야한다. trainspose -> 그러면 서로의 상관관계를 알수 있고, 유사도를 측정하는 방법 중 하나가 된다.
temp = user_item.T.corr()
#6040명의 사람 데이터가 있으니, 계산량이 어마어마하게 많아서 실행시간이 오래걸리는 편이다. 
#얘는 cpu 연산하기 때문에 느린거다. 겨우 6000명 정도지만. 
#만약 일반 쇼핑몰 몇십만명 계산하려면, gpu 연산이 가능하도록 만들어줘야한다. 

In [50]:
temp.loc[1005].nlargest(10)

UserID
1005    1.000000
4132    0.418728
295     0.382294
3684    0.378698
4226    0.374644
5842    0.348435
1693    0.338026
5323    0.332459
2388    0.329556
3674    0.328897
Name: 1005, dtype: float64

In [51]:
temp.loc[1005].sort_values(ascending=False)[1:5]
#sort는 중요하다. 순서 정하기 위해서. 

UserID
4132    0.418728
295     0.382294
3684    0.378698
4226    0.374644
Name: 1005, dtype: float64

In [52]:
rating.set_index(['UserID', 'MovieID']).unstack()

Rating                                               ... Timestamp  \
MovieID   1    2    3    4    5    6    7    8    9    10    ...      3943   
UserID                                                       ...             
1          5.0  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  ...       NaN   
2          NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  ...       NaN   
3          NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  ...       NaN   
4          NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  ...       NaN   
5          NaN  NaN  NaN  NaN  NaN  2.0  NaN  NaN  NaN  NaN  ...       NaN   
...        ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...       ...   
6036       NaN  NaN  NaN  2.0  NaN  3.0  NaN  NaN  NaN  NaN  ...       NaN   
6037       NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  ...       NaN   
6038       NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  ...       NaN   
6039       NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  ...       NaN   
6040       3.0  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  ...       NaN   

                                                      
MovieID 3944 3945 3946 3947 3948 3949 3950 3951 3952  
UserID                                                
1        NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  
2        NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  
3        NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  
4        NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  
5        NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  
...      ...  ...  ...  ...  ...  ...  ...  ...  ...  
6036     NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  
6037     NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  
6038     NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  
6039     NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  
6040     NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  

[6040 rows x 7412 columns]

### 함수형 패러다임으로 가까운 정보 뽑아내는 절차 편하게 하기

In [53]:
def sim_user(user, n):
    #user에 집어넣으면 나하고 가까운 n명 뽑아내는거임.
    return temp.loc[user].sort_values(ascending=False)[1:n]

In [54]:
sim_user(1,5)

UserID
5343    0.405356
5190    0.404446
1481    0.380623
1283    0.379054
Name: 1, dtype: float64

In [55]:
#내가 안본 영화 추천받기 위해서, 저들이 본 영화 뽑아내는 방법
#1번 사람이 본 영화 목록
rating[rating.UserID==1]

,UserID,MovieID,Rating,Timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291
5,1,1197,3,978302268
6,1,1287,5,978302039
7,1,2804,5,978300719
8,1,594,4,978302268
9,1,919,4,978301368


In [56]:
#이번에도 함수로 만들어보기
def user_seen(user):
    return rating[rating.UserID==user]['MovieID'].values
    #user 가 본 영화의 영화ID의 인덱스 제외한 값만!

In [57]:
user_seen(1)

array([1193,  661,  914, 3408, 2355, 1197, 1287, 2804,  594,  919,  595,
        938, 2398, 2918, 1035, 2791, 2687, 2018, 3105, 2797, 2321,  720,
       1270,  527, 2340,   48, 1097, 1721, 1545,  745, 2294, 3186, 1566,
        588, 1907,  783, 1836, 1022, 2762,  150,    1, 1961, 1962, 2692,
        260, 1028, 1029, 1207, 2028,  531, 3114,  608, 1246], dtype=int64)

In [58]:
#합집합으로 여러명이 본 영화도 뽑아낼 수 있다. 중복 제거하고. 
user_seen(5343)
#근데 만약 10명 정보를 얻으려면, 이걸 여러번 쳐야할까?

array([ 588,    1,  593,  594,  595, 2078, 2080, 2081, 2096,  910,  919,
       3408, 3421, 2858,  608, 2137, 1197, 3114, 3578, 1221, 1230, 1617,
       3298,  457, 1343, 3301, 1022, 2762, 1023, 1028, 1029, 3712, 2791,
        527, 2028, 1097], dtype=int64)

### 아니다. 고급 3총사가 있다고 한다,,,
1. query
2. where
3. is in

## is in 
여러개 중에서 이게 있는지 없는지 체크하는데 쓴다고한다.
원래 알고있던 in 이 판다스에서 is in 으로 확장되었다고 한다.

In [59]:
1 in [1,2,3,4]

True

In [60]:
def sim_user(user, n):
    return temp.loc[user].sort_values(ascending=False)[1:n+1].index

In [61]:
sim_user(1,5)

Index([5343, 5190, 1481, 1283, 5705], dtype='int64', name='UserID')

In [62]:
def user_seen(user):
    return rating[rating.UserID==user]['MovieID'].values

In [64]:
rating.UserID.values
#중복이 있는 데이터다 보니 set을 써줘야한다.

array([   1,    1,    1, ..., 6040, 6040, 6040], dtype=int64)

In [65]:
set(rating.UserID.values)

{1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 102,
 103,
 104,
 105,
 106,
 107,
 108,
 109,
 110,
 111,
 112,
 113,
 114,
 115,
 116,
 117,
 118,
 119,
 120,
 121,
 122,
 123,
 124,
 125,
 126,
 127,
 128,
 129,
 130,
 131,
 132,
 133,
 134,
 135,
 136,
 137,
 138,
 139,
 140,
 141,
 142,
 143,
 144,
 145,
 146,
 147,
 148,
 149,
 150,
 151,
 152,
 153,
 154,
 155,
 156,
 157,
 158,
 159,
 160,
 161,
 162,
 163,
 164,
 165,
 166,
 167,
 168,
 169,
 170,
 171,
 172,
 173,
 174,
 175,
 176,
 177,
 178,
 179,
 180,
 181,
 182,
 183,
 184,
 185

##### 1 or 2 or 3 이 개념은
x in [1,2,3] 와 같다.

In [ ]:
#isin 예시
[3,4].isin([1,2,3,4,5])
#벡터화, 행렬화 동시에 처리해주는거다. vetorization... 

In [68]:
rating[(rating.UserID == 5343)|(rating.UserID == 5190)|(rating.UserID == 1481)]
#이렇게 만들면 식으로 만들기 어렵다. userID가 가까운 애들인지 모르니까.

,UserID,MovieID,Rating,Timestamp
245227,1481,2987,5,974755113
245228,1481,2997,4,974838939
245229,1481,1259,4,974752073
245230,1481,588,4,974755113
245231,1481,3000,5,974755337
...,...,...,...,...
884705,5343,3712,2,960681969
884706,5343,2791,4,960682425
884707,5343,527,5,960682228
884708,5343,2028,5,960682252


In [71]:
rating.UserID.isin([5343,5190,1481])

0          False
1          False
2          False
3          False
4          False
           ...  
1000204    False
1000205    False
1000206    False
1000207    False
1000208    False
Name: UserID, Length: 1000209, dtype: bool

In [72]:
rating[rating.UserID.isin([5343,5190,1481])]

,UserID,MovieID,Rating,Timestamp
245227,1481,2987,5,974755113
245228,1481,2997,4,974838939
245229,1481,1259,4,974752073
245230,1481,588,4,974755113
245231,1481,3000,5,974755337
...,...,...,...,...
884705,5343,3712,2,960681969
884706,5343,2791,4,960682425
884707,5343,527,5,960682228
884708,5343,2028,5,960682252


In [73]:
#식으로 만든 버전이다. .... 
rating[rating.UserID.isin(user_seen(1))]
#user 1번이 본 영화

,UserID,MovieID,Rating,Timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291
...,...,...,...,...
553558,3408,3771,4,967422602
553559,3408,1097,3,967423277
553560,3408,1240,5,967421749
553561,3408,1242,5,967422499


In [76]:
rating[rating.UserID.isin(sim_user(1,5))]
#1번과 가까운 5명이 본 영화 목록. 

,UserID,MovieID,Rating,Timestamp
209863,1283,2059,4,974794338
209864,1283,588,4,974793527
209865,1283,1,5,974794375
209866,1283,1407,3,974794241
209867,1283,593,3,974793527
...,...,...,...,...
945317,5705,543,5,958545469
945318,5705,2028,5,958544786
945319,5705,1097,4,958544126
945320,5705,2043,3,958546321


In [74]:
#하지만, 내가 본 영화목록(중복값)은 지워줘야한다. 
#저들이 본영화 - 내가 본영화 : 차집합을 이용해주면 된다.

array([1193,  661,  914, 3408, 2355, 1197, 1287, 2804,  594,  919,  595,
        938, 2398, 2918, 1035, 2791, 2687, 2018, 3105, 2797, 2321,  720,
       1270,  527, 2340,   48, 1097, 1721, 1545,  745, 2294, 3186, 1566,
        588, 1907,  783, 1836, 1022, 2762,  150,    1, 1961, 1962, 2692,
        260, 1028, 1029, 1207, 2028,  531, 3114,  608, 1246], dtype=int64)

In [77]:
def sim_user_seen(user,n):
    u = temp.loc[user].sort_values(ascending=False)[1:n+1].index
    return rating[rating.UserID.isin(u)]['MovieID'].values

In [78]:
set(sim_user_seen(1,5))

{1,
 2,
 7,
 11,
 29,
 34,
 36,
 39,
 46,
 47,
 48,
 74,
 86,
 110,
 150,
 154,
 158,
 168,
 195,
 207,
 236,
 246,
 252,
 260,
 266,
 296,
 303,
 318,
 339,
 351,
 356,
 364,
 377,
 380,
 440,
 446,
 455,
 457,
 480,
 527,
 531,
 539,
 543,
 551,
 587,
 588,
 590,
 592,
 593,
 594,
 595,
 596,
 597,
 605,
 608,
 648,
 661,
 673,
 708,
 709,
 720,
 733,
 741,
 745,
 780,
 783,
 784,
 788,
 832,
 838,
 866,
 902,
 904,
 910,
 912,
 914,
 916,
 917,
 919,
 920,
 923,
 986,
 994,
 1022,
 1023,
 1028,
 1029,
 1030,
 1031,
 1032,
 1033,
 1035,
 1036,
 1043,
 1049,
 1059,
 1060,
 1073,
 1094,
 1095,
 1097,
 1136,
 1148,
 1177,
 1183,
 1188,
 1197,
 1198,
 1201,
 1207,
 1210,
 1213,
 1219,
 1221,
 1223,
 1225,
 1230,
 1234,
 1240,
 1258,
 1259,
 1265,
 1266,
 1270,
 1274,
 1280,
 1282,
 1287,
 1343,
 1345,
 1377,
 1387,
 1393,
 1396,
 1405,
 1407,
 1409,
 1500,
 1515,
 1517,
 1537,
 1541,
 1566,
 1569,
 1573,
 1580,
 1584,
 1593,
 1617,
 1649,
 1680,
 1682,
 1704,
 1721,
 1722,
 1735,
 1748,


In [81]:
import numpy as np

In [82]:
#user가 봤지만, 내가 본거 제외한 함수
def sim_user_seen_except_me(user,n):
    u = temp.loc[user].sort_values(ascending=False)[1:n+1].index
    um = rating[rating.UserID.isin(u)]['MovieID'].values
    my = rating[rating.UserID==user]['MovieID'].values
    return np.setdiff1d(um, my)

In [83]:
sim_user_seen_except_me(3,3)
#3번이랑 가까운 3명이, 3번이 안본영화들로만 리스트

array([   1,    2,    9,   10,   11,   25,   32,   34,   39,   47,  112,
        144,  150,  153,  160,  161,  169,  208,  223,  296,  318,  329,
        339,  349,  356,  361,  362,  367,  368,  380,  421,  440,  455,
        492,  527,  539,  541,  543,  588,  589,  592,  597,  736,  737,
        761,  852,  919,  924,  933,  934, 1011, 1015, 1017, 1028, 1031,
       1073, 1080, 1088, 1090, 1097, 1125, 1127, 1129, 1193, 1199, 1200,
       1206, 1213, 1214, 1220, 1234, 1240, 1246, 1252, 1262, 1267, 1278,
       1287, 1307, 1356, 1372, 1373, 1374, 1375, 1376, 1377, 1381, 1396,
       1459, 1495, 1544, 1552, 1569, 1584, 1595, 1610, 1653, 1663, 1674,
       1676, 1704, 1721, 1747, 1777, 1779, 1784, 1894, 1931, 1954, 1958,
       1962, 2000, 2001, 2005, 2011, 2013, 2028, 2046, 2050, 2054, 2078,
       2080, 2100, 2108, 2133, 2135, 2136, 2140, 2144, 2153, 2161, 2174,
       2236, 2240, 2243, 2268, 2288, 2301, 2302, 2321, 2352, 2366, 2371,
       2405, 2406, 2407, 2414, 2421, 2422, 2423, 24

In [84]:
mi = movie.set_index('MovieID')

In [85]:
mi.loc[[1,3,5]]
#1,3,5영화 알려달라.

,Title,Genres
MovieID,,
1,Toy Story (1995),Animation|Children's|Comedy
3,Grumpier Old Men (1995),Comedy|Romance
5,Father of the Bride Part II (1995),Comedy


In [86]:
mi.loc[sim_user_seen_except_me(3,3)]
#3번과 가까운 3명이 봤지만, 내가 안본 영화들로

,Title,Genres
MovieID,,
1,Toy Story (1995),Animation|Children's|Comedy
2,Jumanji (1995),Adventure|Children's|Fantasy
9,Sudden Death (1995),Action
10,GoldenEye (1995),Action|Adventure|Thriller
11,"American President, The (1995)",Comedy|Drama|Romance
...,...,...
3703,Mad Max 2 (a.k.a. The Road Warrior) (1981),Action|Sci-Fi
3705,Bird on a Wire (1990),Action|Adventure|Romance|Thriller
3793,X-Men (2000),Action|Sci-Fi


In [87]:
#전체 테이블 (data) 붙여놓은거에 set index MovieID라고 하고 lake 라는 이름을 붙여줌.
lake = data.set_index('MovieID')

In [88]:
lake.loc[sim_user_seen_except_me(3,3)]

,UserID,Rating,Timestamp,Title,Genres,Gender,Age,Occupation,Zip-code
MovieID,,,,,,,,,
1,1,5,978824268,Toy Story (1995),Animation|Children's|Comedy,F,1,10,48067
1,18,4,978154768,Toy Story (1995),Animation|Children's|Comedy,F,18,3,95825
1,19,5,978555994,Toy Story (1995),Animation|Children's|Comedy,M,1,10,48073
1,28,3,978985309,Toy Story (1995),Animation|Children's|Comedy,F,25,1,14607
1,44,5,978019369,Toy Story (1995),Animation|Children's|Comedy,M,45,17,98052
...,...,...,...,...,...,...,...,...,...
3948,1919,3,974690242,Meet the Parents (2000),Comedy,M,1,0,94525
3948,826,1,975373226,Meet the Parents (2000),Comedy,F,35,1,80210
3948,1154,4,974865952,Meet the Parents (2000),Comedy,F,1,10,53545


## where
~가 있는지 없는지 (오늘은 안할거임)